# Using the AMPL diet model

In [ ]:
# install dependencies
%pip install -q amplpy pandas

from amplpy import AMPL, ampl_notebook
import pandas as pd

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics

By specifying appropriate data, we can solve any of the linear
programs that correspond to the above model.
Let's begin by using the data from the beginning of this
chapter, which is shown in `AMPL` format in Figure @Tut2@-@diet.dat@.

In [2]:
%%writefile diet.mod

set NUTR;
set FOOD;

param cost {FOOD} > 0;
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];

param n_min {NUTR} >= 0;
param n_max {i in NUTR} >= n_min[i];

param amt {NUTR,FOOD} >= 0;

var Buy {j in FOOD} >= f_min[j], <= f_max[j];

minimize Total_Cost:  sum {j in FOOD} cost[j] * Buy[j];

subject to Diet {i in NUTR}:
   n_min[i] <= sum {j in FOOD} amt[i,j] * Buy[j] <= n_max[i];

Overwriting diet.mod


In [3]:
df_food = pd.DataFrame(
    [
        ['BEEF', 3.19, 0, 100],
        ['CHK', 2.59, 0, 100],
        ['FISH', 2.29, 0, 100],
        ['HAM', 2.89, 0, 100],
        ['MCH', 1.89, 0, 100],
        ['MTL', 1.99, 0, 100],
        ['SPG', 1.99, 0, 100],
        ['TUR', 2.49, 0, 100]
    ],
    columns = ['FOOD', 'cost', 'f_min', 'f_max']
).set_index('FOOD')

df_nutr = pd.DataFrame(
    [
        ['A', 700, 10000],
        ['C', 700, 10000],
        ['B1', 700, 10000],
        ['B2', 700, 10000]
    ],
    columns = ['NUTR', 'n_min', 'n_max']
).set_index('NUTR')

df_amt = pd.DataFrame(
    [
        ['BEEF', 60, 20, 10, 15],
        ['CHK', 8, 0, 20, 20],
        ['FISH', 8, 10, 15, 10],
        ['HAM', 40, 40, 35, 10],
        ['MCH', 15, 35, 15, 15],
        ['MTL', 70, 30, 15, 15],
        ['SPG', 25, 50, 25, 15],
        ['TUR', 60, 20, 15, 10]
    ],
    columns = ['FOOD', 'A', 'C', 'B1', 'B2']
).set_index('FOOD').T

display(df_food)
display(df_nutr)
display(df_amt)

,cost,f_min,f_max
FOOD,,,
BEEF,3.19,0,100
CHK,2.59,0,100
FISH,2.29,0,100
HAM,2.89,0,100
MCH,1.89,0,100
MTL,1.99,0,100
SPG,1.99,0,100
TUR,2.49,0,100


,n_min,n_max
NUTR,,
A,700,10000
C,700,10000
B1,700,10000
B2,700,10000


FOOD,BEEF,CHK,FISH,HAM,MCH,MTL,SPG,TUR
A,60,8,8,40,15,70,25,60
C,20,0,10,40,35,30,50,20
B1,10,20,15,35,15,15,25,15
B2,15,20,10,10,15,15,15,10


The values of
`f_min`
and
`n_min`
are as given originally, while
`f_max`
and
`n_max`
are set, for the time being, to large values that won't affect the
optimal solution.  In the table for
`amt` ,
the notation
`(tr)`
indicates that we
have "transposed" the table so the columns
correspond to the first index (nutrients), and the rows to the second (foods).
Alternatively, we could have changed the model to say
```
param amt {FOOD,NUTR}
```
in which case we would have had to write
`amt[j,i]`
in the
constraint.

Suppose that model and data are stored in the files
`diet.mod`
and
`diet.dat` ,
respectively.  Then `AMPL` is used as follows to read these files and to
solve the resulting linear program:


In [4]:
ampl = AMPL()
ampl.read('diet.mod')

ampl.set_data(df_nutr, 'NUTR')
ampl.set_data(df_food, 'FOOD')

ampl.param['amt'] = df_amt

ampl.solve(solver='highs')

ampl.display('Buy')

HiGHS 1.6.0: HiGHS 1.6.0: optimal solution; objective 88.2
2 simplex iterations
0 barrier iterations
 
Buy [*] :=
BEEF   0
 CHK   0
FISH   0
 HAM   0
 MCH  46.6667
 MTL   0
 SPG   0
 TUR   0
;



Naturally, the result is the same as before.

Now suppose that we want to make the following enhancements.  To
promote variety, the weekly diet must contain between 2 and 10 packages of each
food.  The amount of sodium and calories in each package is also given;
total sodium must not exceed 40,000 mg, and total calories must be
between 16,000 and 24,000.  All of these changes can be made
through a few modifications to the data, as shown in Figure @Tut2@-@diet2.dat@.

In [5]:
df_food2 = pd.DataFrame(
    [
        ['BEEF', 3.19, 2, 10],
        ['CHK', 2.59, 2, 10],
        ['FISH', 2.29, 2, 10],
        ['HAM', 2.89, 2, 10],
        ['MCH', 1.89, 2, 10],
        ['MTL', 1.99, 2, 10],
        ['SPG', 1.99, 2, 10],
        ['TUR', 2.49, 2, 10]
    ],
    columns = ['FOOD', 'cost', 'f_min', 'f_max']
).set_index('FOOD')

df_nutr2 = pd.DataFrame(
    [
        ['A', 700, 20000],
        ['C', 700, 20000],
        ['B1', 700, 20000],
        ['B2', 700, 20000],
        ['NA', 0, 40000],
        ['CAL', 16000, 24000],
    ],
    columns = ['NUTR', 'n_min', 'n_max']
).set_index('NUTR')

df_amt2 = pd.DataFrame(
    [
        ['BEEF', 60, 20, 10, 15, 938, 295],
        ['CHK', 8, 0, 20, 20, 2180, 770],
        ['FISH', 8, 10, 15, 10, 945, 440],
        ['HAM', 40, 40, 35, 10, 278, 430],
        ['MCH', 15, 35, 15, 15, 1182, 315],
        ['MTL', 70, 30, 15, 15, 896, 400],
        ['SPG', 25, 50, 25, 15, 1329, 370],
        ['TUR', 60, 20, 15, 10, 1397, 450]
    ],
    columns = ['FOOD', 'A', 'C', 'B1', 'B2', 'NA', 'CAL']
).set_index('FOOD').T

display(df_food2)
display(df_nutr2)
display(df_amt2)

,cost,f_min,f_max
FOOD,,,
BEEF,3.19,2,10
CHK,2.59,2,10
FISH,2.29,2,10
HAM,2.89,2,10
MCH,1.89,2,10
MTL,1.99,2,10
SPG,1.99,2,10
TUR,2.49,2,10


,n_min,n_max
NUTR,,
A,700,20000
C,700,20000
B1,700,20000
B2,700,20000
NA,0,40000
CAL,16000,24000


FOOD,BEEF,CHK,FISH,HAM,MCH,MTL,SPG,TUR
A,60,8,8,40,15,70,25,60
C,20,0,10,40,35,30,50,20
B1,10,20,15,35,15,15,25,15
B2,15,20,10,10,15,15,15,10
NA,938,2180,945,278,1182,896,1329,1397
CAL,295,770,440,430,315,400,370,450


Putting this new data in file
`diet2.dat` ,
we can run `AMPL` again:

In [6]:
ampl = AMPL()
ampl.read('diet.mod')

ampl.set_data(df_nutr2, 'NUTR')
ampl.set_data(df_food2, 'FOOD')

ampl.param['amt'] = df_amt2

ampl.solve(solver='highs')

HiGHS 1.6.0: HiGHS 1.6.0: infeasible problem
4 simplex iterations
0 barrier iterations
 


The message
`infeasible problem`
tells us that we have constrained the diet too
tightly; there is no way that all of the restrictions can be satisfied.

`AMPL` lets us examine a variety of values produced by a solver
as it attempts to find a solution.
In Chapter @Tut1@, we used marginal (or dual) values to
investigate the sensitivity of an optimum solution to changes in the constraints.
Here there is no optimum, but the solver does return the last solution that it
found while attempting to satisfy the constraints.  We can look for the source of
the infeasibility by displaying some values associated with this solution:

In [7]:
ampl.display('Diet.lb', 'Diet.body', 'Diet.ub')

:   Diet.lb Diet.body Diet.ub    :=
A       700      0      20000
B1      700      0      20000
B2      700      0      20000
C       700      0      20000
CAL   16000      0      24000
NA        0      0      40000
;



For each nutrient,
`Diet.body`
.ix constraint~body
is the sum of the terms
`amt[i,j]
*
Buy[j]`
in the constraint
`Diet[i]` .
The
`Diet.lb`
and
`Diet.ub`
values are the "lower bounds" and "upper bounds" on the sum in
`Diet[i]`
&ndash; 
in this case,
just the values
`n_min[i]`
and
`n_max[i]` .
We can see that the diet returned by the solver
does not supply enough vitamin B2, while the amount of sodium
NA ) (
has reached its upper bound.

At this point, there are two obvious choices: we could require less B2 or
we could allow more sodium.
If we try the latter, and relax the sodium limit to 50,000 mg,
a feasible solution becomes possible:

In [8]:
ampl.param['n_max']["NA"] = 50000
ampl.solve()
ampl.display('Buy')

HiGHS 1.6.0: HiGHS 1.6.0: optimal solution; objective 118.0594032
1 simplex iterations
0 barrier iterations
 
Buy [*] :=
BEEF   5.36061
 CHK   2
FISH   2
 HAM  10
 MCH  10
 MTL  10
 SPG   9.30605
 TUR   2
;



This is at least a start toward a palatable diet, although we have to spend
$\$$118.06, compared to $\$$88.20 for the original, less restricted case.
Clearly it would be easy, now that the model is set up, to try many other
possibilities.
(Section @Command@.@cmd-moddata@ describes ways to quickly change the data
and re-solve.)

One still disappointing aspect of the solution is the need to
buy 5.36061 packages of beef, and 9.30605 of spaghetti.  How can we find the
best possible solution in terms of whole packages?  You might think that we
could simply round the
optimal values to whole numbers &ndash; or integers, as they're often called in the
context of optimization &ndash; but it is not so easy to do so in a feasible
way.  Using `AMPL` to modify the reported solution, we can observe that rounding up
to 6 packages of beef and 10 of spaghetti, for example, will violate
the sodium limit:

In [9]:
ampl.var['Buy']['BEEF'] = 6
ampl.var['Buy']['SPG'] = 10
ampl.display('Diet.lb', 'Diet.body', 'Diet.ub')

:   Diet.lb Diet.body Diet.ub    :=
A       700     2012    20000
B1      700     1060    20000
B2      700      720    20000
C       700     1730    20000
CAL   16000    20240    24000
NA        0    51522    50000
;



(The
`let`
statement, which permits modifications of data,
is described in Section @Command@.@cmd-moddata@.)
You can similarly check that rounding the solution down to 5 of beef and 9 of
spaghetti will provide insufficient
vitamin B2.  Rounding one up and the other down doesn't work either.
With enough experimenting you can find a nearby all-integer solution that does satisfy the
constraints, but still you will have no guarantee that it is the
least-cost all-integer solution.

`AMPL` does provide for putting the integrality restriction
directly into the declaration of the variables:
```
var Buy {j in FOOD} integer >= f_min[j], <= f_max[j];
```
This will only help, however, if you use
a solver that can deal with problems whose variables must be integers.
For this, we turn to
`CPLEX` ,
a solver that can
handle these so-called integer programs.
If we add
`integer`
to the declaration of variable
`Buy`
as above, save the resulting model in the file
`dieti.mod` ,
and add the higher sodium limit to
`diet2a.dat` ,
then we can re-solve as follows:

In [10]:
%%writefile dieti.mod

set NUTR;
set FOOD;

param cost {FOOD} > 0;
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];

param n_min {NUTR} >= 0;
param n_max {i in NUTR} >= n_min[i];

param amt {NUTR,FOOD} >= 0;

var Buy {j in FOOD} integer >= f_min[j], <= f_max[j];

minimize Total_Cost:  sum {j in FOOD} cost[j] * Buy[j];

subject to Diet {i in NUTR}:
   n_min[i] <= sum {j in FOOD} amt[i,j] * Buy[j] <= n_max[i];

Overwriting dieti.mod


In [11]:
df_nutr2a = pd.DataFrame(
    [
        ['A', 700, 20000],
        ['C', 700, 20000],
        ['B1', 700, 20000],
        ['B2', 700, 20000],
        ['NA', 0, 50000],
        ['CAL', 16000, 24000],
    ],
    columns = ['NUTR', 'n_min', 'n_max']
).set_index('NUTR')

In [12]:
ampl = AMPL()
ampl.read('dieti.mod')

ampl.set_data(df_nutr2a, 'NUTR')
ampl.set_data(df_food2, 'FOOD')

ampl.param['amt'] = df_amt2

ampl.solve(solver='highs')

ampl.display('Buy')

HiGHS 1.6.0: HiGHS 1.6.0: optimal solution; objective 119.3
12 simplex iterations
1 branching nodes
 
Buy [*] :=
BEEF   9
 CHK   2
FISH   2
 HAM   8
 MCH  10
 MTL  10
 SPG   7
 TUR   2
;



Since integrality is an added constraint, it is no surprise that the best integer
solution costs about $\$$1.24 more than the best "continuous" one.  But the difference
between the diets is unexpected; the amounts of 3 foods change, each by two or more
packages.  In general,
integrality and other "discrete" restrictions make solutions for a model much
harder to find.
We discuss this at length in Chapter @Integer@.